In [2]:
# !pip install pandas sas7bdat pyreadstat
import pandas as pd
import sas7bdat, signal, pyreadstat, pickle
from toolbox import *

In [28]:
import pandas as pd

# Define the path using a raw string and ensuring proper path formatting
file_path = r"..\own_data_objects\melted_tables\melted_table_0.csv"
answers = unpickle("../own_data_objects/answers.pkl")

# Read the CSV file while specifying the data type for the 'Answer' column as string
df = pd.read_csv(file_path)
df = compare_years_delta(df, 2000, 2010)
df1 = df.copy()

# print(df.head(3))

def modify_answers(df, answers):
    def get_answer(row):
        question = row.iloc[0]
        answer = row.iloc[1]
        if question in answers and str(int(answer)) in answers[question]:
            return answers[question][str(int(answer))]
        else:
            return answer

    df.iloc[:, 1] = df.apply(get_answer, axis=1)

def modify_answers1(df, answers):
    # Check if the expected columns are present
    if 'Question' not in df.columns or 'Answer' not in df.columns:
        raise ValueError("DataFrame must contain 'Question' and 'Answer' columns")

    # Ensure the Answer column is treated as a string for consistent key comparison
    df['Answer'] = df['Answer'].apply(lambda x: str(int(x)).strip() if pd.notna(x) else x)

    # Using apply to process each row; referencing columns by names
    def get_answer(row):
        question = row['Question']
        answer = row['Answer']
        if question in answers and answer in answers[question]:
            return answers[question][answer]
        return answer

    df['Answer'] = df.apply(get_answer, axis=1)
    return df


modify_answers1(df1, answers)
print(df1.head(4))

     Question         Answer  2000 Percentage  2010 Percentage  \
77   CONFINAN     Hardly any        14.448669        41.348974   
162  FINALTER  Getting worse        16.916488        36.989670   
458  NATHEALY       Too much         6.217996        21.105528   
701       TAX    About right        32.523998        47.001621   

     2010-2000 Delta  
77         26.900304  
162        20.073182  
458        14.887532  
701        14.477623  


In [77]:
answers['HOMOSEX']

{'1203': 'NOT APPLICABLE I', '9': 'SKIPPED ON WEB S'}

In [2]:
import pandas as pd

# Example DataFrame setup
data = {
    'Question': ['Q1', 'Q2', 'Q3'],
    'Answer': [100, 200, 300]
}
df = pd.DataFrame(data)

# Example 'answers' dictionary
answers = {
    'Q1': 'Answer for Q1',
    'Q2': 'Answer for Q2',
    'Q3': 'Answer for Q3'
}

# Update the 'Answer' column based on the 'Question' column
df['Answer'] = df.apply(lambda row: f"{answers[row['Question']]} [{row['Answer']}]", axis=1)

print(df)


  Question               Answer
0       Q1  Answer for Q1 [100]
1       Q2  Answer for Q2 [200]
2       Q3  Answer for Q3 [300]


In [3]:
# !pip install pandas sas7bdat pyreadstat
import pandas as pd
import sas7bdat, signal, pyreadstat, pickle
from toolbox import *

import os

file_path = "C:\\Users\\justi\\Downloads\\GSS_sas\\gss7222_r3a.sas7bdat"

if os.path.exists(file_path):
    print("The file exists.")
else:
    print("The file does not exist.")

data_path = r"C:\Users\justi\Downloads\GSS_sas\gss7222_r3a.sas7bdat"
df, meta = pyreadstat.read_sas7bdat(data_path)

The file exists.


In [16]:
# # ["WLTHBLKS"]
# d = vars(meta)
# for k in d.keys():
#     print("key")
#     print(k)
#     print("key value")
#     print(d[k])
# print(d.keys())
# print(d['variable_value_labels'])
# print(d['value_labels'])

key
column_names
key value
['YEAR', 'ID', 'WRKSTAT', 'HRS1', 'HRS2', 'EVWORK', 'OCC', 'PRESTIGE', 'WRKSLF', 'WRKGOVT', 'COMMUTE', 'INDUSTRY', 'OCC80', 'PRESTG80', 'INDUS80', 'INDUS07', 'OCCONET', 'FOUND', 'OCC10', 'OCCINDV', 'OCCSTATUS', 'OCCTAG', 'PRESTG10', 'PRESTG105PLUS', 'INDUS10', 'INDSTATUS', 'INDTAG', 'MARITAL', 'MARTYPE', 'AGEWED', 'DIVORCE', 'WIDOWED', 'SPWRKSTA', 'SPHRS1', 'SPHRS2', 'SPEVWORK', 'COWRKSTA', 'COWRKSLF', 'COEVWORK', 'COHRS1', 'COHRS2', 'SPOCC', 'SPPRES', 'SPWRKSLF', 'SPIND', 'SPOCC80', 'SPPRES80', 'SPIND80', 'SPOCC10', 'SPOCCINDV', 'SPOCCSTATUS', 'SPOCCTAG', 'SPPRES10', 'SPPRES105PLUS', 'SPIND10', 'SPINDSTATUS', 'SPINDTAG', 'COOCC10', 'COIND10', 'PAOCC16', 'PAPRES16', 'PAWRKSLF', 'PAIND16', 'PAOCC80', 'PAPRES80', 'PAIND80', 'PAOCC10', 'PAOCCINDV', 'PAOCCSTATUS', 'PAOCCTAG', 'PAPRES10', 'PAPRES105PLUS', 'PAIND10', 'PAINDSTATUS', 'PAINDTAG', 'MAOCC80', 'MAPRES80', 'MAWRKSLF', 'MAIND80', 'MAOCC10', 'MAOCCINDV', 'MAOCCSTATUS', 'MAOCCTAG', 'MAPRES10', 'MAPRES105PLUS

Step 1: Get list of df ready for primetime, first for general, then each for specific section of country

In [3]:
data_path = r'C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\GSS_sas\gss7222_r3.sas7bdat'
df, meta = pyreadstat.read_sas7bdat(data_path)

############# cleaning df #############
# df is the original data - each column header is a question, each row is respondents answers
# need to change the values in the Age column to 0 if between 18-34
# 1 if between 35-49, 2 if between 50-64, 3 if 65+
df['AGE'] = df['AGE'].apply(lambda x: 0 if x < 35 else 1 if x < 50 else 2 if x < 65 else 3)
# do same for degree, where is 3 or 4, make it 2
df['DEGREE'] = df['DEGREE'].apply(lambda x: 2 if x >= 2 else x)
# for partyid, if 0-1 do 0, 2-4,7 do 1, 5-6 do 2
df['PARTYID'] = df['PARTYID'].apply(lambda x: 0 if x < 2 else 1 if x < 5 or x == 7 else 2)
# leaving sex and race alone, those go from 1-2 and 1-3 respectively

############# creating df_list #############
df_list = [df]
age_list = [df[df['AGE'] == age] for age in range(4)]
degree_list = [df[df['DEGREE'] == degree] for degree in range(3)]
partyid_list = [df[df['PARTYID'] == partyid] for partyid in range(3)]
sex_list = [df[df['SEX'] == sex] for sex in range(1, 3)]
race_list = [df[df['RACE'] == race] for race in range(1, 4)]
df_list.extend(age_list)
df_list.extend(degree_list)
df_list.extend(partyid_list)
df_list.extend(sex_list)
df_list.extend(race_list)



            

Step 2: Get melted tables list for each df. First get Year and Question pairings from general one, with min_answers, then have melted tables list - each question, how that demographic answered over time. 

These tables are saved in system, and need to be called upon quickly for Step 3

In [4]:
############# params for creating first melted table #############
# df, id_vars, exclude_columns, include_columns, year, min_answers
paradata_keywords = read_file_lines(r"C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\GSS_sas\Paradata_variables.txt")
df = df; id_vars=['YEAR']; exclude_columns=['ID']+(paradata_keywords); include_columns=[]
year = 2000; min_answers = 850

############# creating first melted table, and params to filter future ones #############
# melted table cols --- id_vars (YEAR), Question, Answer, Percentage, Num_Answers
melted_list = []
melted_list.append(process_survey_data(df, id_vars=id_vars, exclude_columns=exclude_columns, include_columns=include_columns, year=year,min_answers=min_answers))
filter_df = melted_list[0]["YEAR", "Question"]

############# creating future melted tables #############
for i in range(1, len(df_list)):
    df = df_list[i]
    # min_answers = 0 because already got YEAR, Question pairings of interest from first df
    melted_df = process_survey_data(df, id_vars=id_vars, 
        exclude_columns=exclude_columns, include_columns=include_columns, 
        year=year, min_answers=0)
    filtered_df = filter_dataframe(melted_df, filter_df)
    melted_list.append(filtered_df)


Step 3: Can put any of the melted dfs into compare_years_delta, and get data table sorted by delta

In [1]:
############# given melted_df, and years, get sorted by biggest delta #############
# final_table = compare_years_delta(melted_table, 2010, 2018)
# print(final_table.head(50))

############# vars so can find correct melted table for the website #############
place = {"AGE":1, "DEGREE":5, "PARTYID":8,"SEX":11,"RACE":13}
naming = ["The Country", "18-34", "35-49", "50-64", "65+", "No High School", 
          "High School", "College+", "Democrat", "Independent/ Other", 
          "Republican", "Male", "Female", "White", "Black", "Other"]

In [2]:
naming.index("The Country")

0

In [ ]:
# Define the segment and its corresponding file indices based on the new system
segment_files = {
    "Whole Country": "melted_table_0.csv",
    "Age": ["melted_table_1.csv", "melted_table_2.csv", "melted_table_3.csv", "melted_table_4.csv"],
    "Degree": ["melted_table_5.csv", "melted_table_6.csv", "melted_table_7.csv"],
    "Partyid": ["melted_table_8.csv", "melted_table_9.csv", "melted_table_10.csv"],
    "Sex": ["melted_table_11.csv", "melted_table_12.csv"],
    "Race": ["melted_table_13.csv", "melted_table_14.csv", "melted_table_15.csv"]
}

# Sub-segment options based on selection
sub_segment_options = {
    "Age": ["18-34", "35-49", "50-64", "65+"],
    "Degree": ["No High School", "High School", "College+"],
    "Partyid": ["Democrat", "Independent/ Other", "Republican"],
    "Sex": ["Male", "Female"],
    "Race": ["White", "Black", "Other"]
}

@callback(
    Output('table-container', 'children'),
    [Input('segment-dropdown', 'value'),
     Input('sub-segment-dropdown', 'value'),
     Input('start-year-dropdown', 'value'),
     Input('end-year-dropdown', 'value'),
     Input('num-rows-input', 'value')]
)
def update_output(segment, sub_segment, start_year, end_year, num_rows):
    print(sub_segment)
    filenames = segment_files[segment]
    # If there are multiple files for the segment, choose based on the sub_segment
    if isinstance(filenames, list):
        # Find the index of the sub_segment in the options list
        sub_index = sub_segment_options[segment].index(sub_segment)
        filename = filenames[sub_index]
    else:
        filename = filenames  # Use single filename directly if not a list

    print(f"Loading data from file: {filename}")
    df = pd.read_csv(f"own_data_objects/melted_tables/{filename}")
    print(f"Data loaded, performing year comparison for years {start_year} and {end_year}")
    df = compare_years_delta(df, int(start_year), int(end_year))
    df.iloc[:, -3:] = df.iloc[:, -3:] / 100
    modify_answers(df, answers)

    # Generate tooltip data only for the first column
    tooltip_data = [
        {df.columns[0]: labels.get(df.at[idx, df.columns[0]], '')}
        for idx in df.index
    ]

    return dash_table.DataTable(
        data=df.to_dict('records'),
        columns=[
            {'name': i, 'id': i, 'type': 'numeric', 'format': Format(precision=1, scheme=Scheme.percentage)} if i in df.columns[-3:] 
            else {'name': i, 'id': i}
            for i in df.columns
        ],
        tooltip_data=tooltip_data,
        page_size=num_rows,
        style_table={'overflowX': 'auto'},
        style_cell={
            "text-align": "center",
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
            'maxWidth': 0,
        },
        tooltip_delay=0,
        tooltip_duration=None
    )


NOW TO PERFECT THE LABELS AND ANSWERS
1) get all labels
2) get other labels
3) complete labels, answers
4) save labels, answers in pickle format 

In [3]:
#### get all labels ####

# code to get melted_table for labels and answers
data_path = r'C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\GSS_sas\gss7222_r3.sas7bdat'
df, meta = pyreadstat.read_sas7bdat(data_path)

paradata_keywords = read_file_lines(r"C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\GSS_sas\Paradata_variables.txt")
df = df; id_vars=['YEAR']; exclude_columns=['ID']+(paradata_keywords); include_columns=[]
year = 2000; min_answers = 850

melted_table = process_survey_data(df, id_vars=id_vars, exclude_columns=exclude_columns, include_columns=include_columns, year=year,min_answers=min_answers)
print(melted_table.head(50))

      YEAR  Question  Answer  Percentage  Num_Answers
0   2000.0     ABANY     1.0   39.875566         1768
1   2000.0     ABANY     2.0   60.124434         1768
2   2000.0  ABDEFECT     1.0   78.748591         1774
3   2000.0  ABDEFECT     2.0   21.251409         1774
4   2000.0    ABHLTH     1.0   88.515406         1785
5   2000.0    ABHLTH     2.0   11.484594         1785
6   2000.0  ABNOMORE     1.0   40.677966         1770
7   2000.0  ABNOMORE     2.0   59.322034         1770
10  2000.0    ABPOOR     1.0   42.244783         1773
11  2000.0    ABPOOR     2.0   57.755217         1773
12  2000.0    ABRAPE     1.0   80.560320         1749
13  2000.0    ABRAPE     2.0   19.439680         1749
14  2000.0  ABSINGLE     1.0   39.105829         1767
15  2000.0  ABSINGLE     2.0   60.894171         1767
18  2000.0    ADULTS     1.0   34.067496         2815
19  2000.0    ADULTS     2.0   53.676732         2815
20  2000.0    ADULTS     3.0    8.987567         2815
21  2000.0    ADULTS     4.0

In [19]:
#### get list_other_labels ####

list_all_labels = melted_table['Question'].tolist()
# get current labels and answers
# Unpickle answers.pkl
with open('../own_data_objects/answers.pkl', 'rb') as file:
    current_answers = pickle.load(file)

# Unpickle labels.pkl
with open('../own_data_objects/labels.pkl', 'rb') as file:
    current_labels = pickle.load(file)

# compare this to labels currently have, just have list what do not have (missing_labels)
# code
list_current_labels = list(current_labels.keys())
list_other_labels = list(set(list_all_labels) - set(list_current_labels))

In [22]:
#### complete labels, and answers ####
def parse3(variable):
    whole_filepath = '../own_data_objects/GSS2022_whole.txt'  # Path to the whole data file
    variable_tag = f'[VAR: {variable}]'  # Format the variable tag to look for

    with open(whole_filepath, 'r', encoding='utf-8') as file:  # Open the file with UTF-8 encoding
        capture = False  # Flag to start capturing data after finding the variable
        after_all = False  # Flag to start recording after "ALL"
        answers = {}  # Dictionary to store the response codes and their descriptions
        
        for line in file:
            line = line.strip()  # Strip any leading/trailing whitespace
            
            if after_all:
                if 'Not applicable' in line:  # Stop capturing on this line
                    break
                # Split the line to separate the description from the codes
                import re
                match = re.split(r'(\d+)', line, 1)  # Split by the first number
                if len(match) >= 2:
                    description = match[0].strip()
                    code = match[1].strip()
                    answers[code] = description
                    # debugging
                    # print(f"Captured: {code} -> {description}")  # Debug output
                else:
                    # debugging
                    # print(f"Skipping line: {line}")  # Debug output for non-data lines
                    pass

            if capture and "ALL" in line:  # This marks the start of the actual data capture
                after_all = True

            if variable_tag in line:  # Check for the specific variable line
                # print(f"Found variable line: {line}")  # Debug output
                capture = True  # Set capture flag to true to start capturing data

    # print("Final captured answers:", answers)  # Output final dictionary
    return answers  # Return the dictionary of response codes and descriptions

def parse2(label):
    index_filepath = '../own_data_objects/GSS_index.txt'  # Path to the index file
    with open(index_filepath, 'r') as file:  # Open the file in read mode
        for line in file:  # Iterate over each line in the file
            if line.startswith(label + " "):  # Check if the line starts with the label followed by a space
                return line.strip().split(' ', 1)[1]  # Split the line at the first space and return the description part
    return None  # Return None if label is not found


# fill in missing labels
for label in list_other_labels:
    value = parse2(label)
    current_labels[label] = value

################################################

# fill in missing answers
for label in list_other_labels:
    value = parse3(label)
    current_answers[label] = value

In [23]:
print(current_labels["ABANY"])
print(current_answers["ABANY"])
print(current_labels["TRDUNION"])
print(current_answers["TRDUNION"])

(Please tell me whether or not you think it should be possible for a pregnant woman to obtain a legal abortion if. . .) If the woman wants it for any reason?
{'1': 'YES', '2': 'NO'}
WORKERS NEED STRONG UNIONS
{'1': 'Strongly agree', '2': 'Agree', '3': 'Disagree', '4': 'Strongly disagree', '8': "Don't know", '9': 'No answer'}


In [30]:
# give me df where sex column is not 1 or 2, just give me that column
test_sex = df[(df['SEX'] != 1) & (df['SEX'] != 2)]['SEX']
test_sex

64816   NaN
64817   NaN
64852   NaN
64930   NaN
64997   NaN
         ..
71080   NaN
71085   NaN
71263   NaN
71271   NaN
71329   NaN
Name: SEX, Length: 112, dtype: float64

In [24]:
######## CLOSE AND SAVE LABELS AND ANSWERS ########

labels, answers = current_labels, current_answers

# Set the directory path where the files will be saved
directory_path = r"C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\own_data_objects"

# File paths
labels_file_path = os.path.join(directory_path, 'labels.pkl')
answers_file_path = os.path.join(directory_path, 'answers.pkl')

# Save the 'labels' dictionary
with open(labels_file_path, 'wb') as file:
    pickle.dump(labels, file)

# Save the 'answers' dictionary
with open(answers_file_path, 'wb') as file:
    pickle.dump(answers, file)


In [28]:
naming = ["The Country", "18-34", "35-49", "50-64", "65+", 
          "No High School", "High School", "College+", "Democrat", "Independent/ Other", 
          "Republican", "Male", "Female", "White", "Black", 
          "Other"]
naming.index("Male")

11

In [19]:
import re

def parse4(variable):
    whole_filepath = '../own_data_objects/GSS2022_whole.txt'  # Path to the whole data file
    variable_tag = f'[VAR: {variable}]'  # Format the variable tag to look for

    with open(whole_filepath, 'r', encoding='utf-8') as file:
        capture = False  # Flag to start capturing data after finding the variable
        after_all = False  # Flag to start recording after "ALL"
        answers = {}  # Dictionary to store the response codes and their descriptions
        
        for line in file:
            line = line.strip()  # Strip any leading/trailing whitespace
            
            if after_all:
                if 'Not applicable' in line:  # Stop capturing on this line
                    break
                # Extract response descriptions and codes; expected format is like:
                # "Always wrong 1 6168 271 ...", capturing description and the first number
                match = re.match(r'^(.+?)\s+(\d+)', line)
                if match:
                    description, code = match.groups()
                    answers[code] = description
                    # Debug output
                    # print(f"Captured: {code} -> {description}")
                else:
                    # Debug output for non-data lines
                    # print(f"Skipping line: {line}")

            if capture and "ALL" in line:  # This marks the start of the actual data capture
                after_all = True

            if variable_tag in line:  # Check for the specific variable line
                capture = True  # Set capture flag to true to start capturing data

    return answers  # Return the dictionary of response codes and descriptions

# Usage
variable = 'HOMOSEX'
answers = parse3(variable)
print("Final captured answers:", answers)


Final captured answers: {'1': 'Always wrong', '2': 'Almost always wrong', '3': 'Wrong only sometimes', '4': 'Not wrong at all', '5': 'Other (SPECIFY)', '8': "Don't know", '9': 'No answer'}


In [1]:
import pickle

# unpickle answers.pkl
with open('../own_data_objects/answers.pkl', 'rb') as file:
    current_answers = pickle.load(file)

# Unpickle labels.pkl
with open('../own_data_objects/labels.pkl', 'rb') as file:
    labels = pickle.load(file)

In [2]:
current_answers["ENDSMEET"]

{'1': 'Essential',
 '2': 'Very important',
 '3': 'Fairly important',
 '4': 'Not very important',
 '5': 'Not important at all',
 '8': "Can't choose",
 '9': 'No answer',
 '0': 'Not Applicable',
 '2264': 'Page',
 '1982': '1972-82',
 '2265': 'Page',
 '1992': 'REMARKS: The'}

In [52]:
# Set the directory path where the files will be saved
directory_path = r"C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\own_data_objects"

# File paths
labels_file_path = os.path.join(directory_path, 'labels.pkl')
answers_file_path = os.path.join(directory_path, 'answers.pkl')

# Save the 'answers' dictionary
with open(answers_file_path, 'wb') as file:
    pickle.dump(a, file)

In [49]:
a['NATSPAC'],a['NATSPACY'] = {}, {}
a['NATSPAC']['2'] = 'About right'
a['NATSPAC']['3'] = 'Too much'
a['NATSPACY']['2'] = 'About right'
a['NATSPACY']['3'] = 'Too much'
a['NATSPAC']['1'] = 'Too little'
a['NATSPACY']['1'] = 'Too little'

In [51]:
# for question in answers.keys():
#     answers[question] = {
#         code: value.split('(')[0].strip()
#         for code, value in answers[question].items()
#     }
a['HOMOSEX']

{'1': 'Always wrong',
 '2': 'Almost always wrong',
 '3': 'Wrong only sometimes',
 '7': 'Wrong only sometimes',
 '4': 'Not wrong at all',
 '5': 'Other',
 '8': "Don't know"}

In [34]:
import re

def clean_descriptions(answers):
    """
    Cleans descriptions in a dictionary of dictionaries by removing any content
    within parentheses (including the parentheses themselves).

    Args:
    answers (dict): A dictionary of dictionaries with descriptions to be cleaned.

    Returns:
    dict: A dictionary with the cleaned descriptions, or an empty dict if input is incorrect.
    """
    if not isinstance(answers, dict):
        print("Input is not a dictionary.")
        return {}
    
    cleaned_answers = {}
    for key, subdict in answers.items():
        if not isinstance(subdict, dict):
            print(f"Value for key '{key}' is not a dictionary.")
            continue
        
        cleaned_subdict = {}
        for subkey, description in subdict.items():
            if not isinstance(description, str):
                print(f"Description for subkey '{subkey}' under key '{key}' is not a string.")
                continue
            # Regex to remove content within parentheses
            cleaned_description = re.sub(r'\s*\([^)]*\)', '', description)
            cleaned_subdict[subkey] = cleaned_description
        cleaned_answers[key] = cleaned_subdict
    
    return cleaned_answers


cleaned_answers = clean_descriptions(current_answers)


In [3]:
import os
source_file_path = r"C:\Users\justi\Dropbox\Work\Job Search\Other\GSS\own_data_objects\GSS2022_whole.txt"
output_directory = r"C:\Users\justi\Downloads\for_answers"
# Create the directory if it does not exist
os.makedirs(output_directory, exist_ok=True)
def process_file():
    with open(source_file_path, 'r', encoding='utf-8') as file:
        content = []
        current_var = None

        for line in file:
            if '[VAR:' in line:
                if content and current_var:
                    # Save the previous section to a file
                    with open(os.path.join(output_directory, f"{current_var}.txt"), 'w', encoding='utf-8') as output_file:
                        output_file.write(''.join(content))
                    content = []
                
                # Extract new variable name
                current_var = line.split('[VAR:')[1].split(']')[0].strip()

            if current_var:
                content.append(line)

        # Handle the last section
        if content and current_var:
            with open(os.path.join(output_directory, f"{current_var}.txt"), 'w', encoding='utf-8') as output_file:
                output_file.write(''.join(content))
process_file()


In [5]:
answers["HOMOSEX"]

NameError: name 'answers' is not defined

In [16]:
import os

# Define the directory containing the files
directory = r'C:\Users\justi\Downloads\for_answers'

# Function to process each file and extract the desired data
def process_files(directory):
    data_dict = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            key = filename[:-4]  # Remove the '.txt' from the filename for the key
            data_dict[key] = {}
            stop = False

            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    if line.startswith("Page") or "REMARKS:" in line:
                        continue  # Skip lines that start with "Page" or contain "REMARKS:"

                    # Split line into parts and examine each part
                    parts = line.strip().split()
                    for i, part in enumerate(parts):
                        if part in {'1', '2', '3', '4', '5', '6', '7', '8'} and i > 0:  # Ensure it's not the first part
                            label = ' '.join(parts[:i])  # Everything before the digit
                            data_dict[key][part] = label
                        elif part == '9':
                            stop = True  # Stop processing this file when "9" is found
                            break
                    if stop:
                        break

    return data_dict

# Call the function and assign the result to a variable
result_dictionary = process_files(directory)

# Display the resulting dictionary
result_dictionary


{'ABANY': {'1': 'Yes', '2': 'No', '8': "Don't know"},
 'ABCARE': {'1': 'Very concerned',
  '2': 'Somewhat concerned',
  '3': 'Not very concerned',
  '4': 'Not concerned at all',
  '8': "Don't know",
  '5': "Don't know 8 0 0 5 0 0 0 0 0 0 0 0 0 0 0"},
 'ABCHOOSE': {'1': 'Strongly agree',
  '2': 'Agree',
  '3': 'Neither agree nor disagree',
  '4': 'Disagree',
  '5': 'Strongly disagree',
  '8': "Can't choose"},
 'ABCON1': {},
 'ABCON2': {'2': 'No answer 99 1',
  '1': 'No answer 99',
  '3': 'No answer 99 1 2 0 0 0 0 0 0 0 0 0 0 0 0'},
 'ABCON3': {'5': '21-30 24',
  '1': 'No answer 99',
  '8': '41-50 24',
  '2': 'No answer 99 1',
  '3': 'No answer 99 1 2 0 0 0 0 0 0 0 0 0 0 0 0'},
 'ABDEFCT1': {'1': 'Definitely should allow it',
  '2': 'Probably should allow it',
  '3': 'Probably should not allow it',
  '4': 'Definitely should not allow it',
  '8': "Can't choose"},
 'ABDEFCTW': {'1': 'Always wrong',
  '2': 'Almost always wrong',
  '3': 'Wrong only sometimes',
  '4': 'Not wrong at all',
  '8

In [12]:
result_dictionary["ENDSMEET"]

{'1': 'Very difficult',
 '2': 'Fairly difficult',
 '3': 'Neither easy nor difficult',
 '4': 'Fairly easy',
 '5': "Don't know 8 0 0 0 0 0 0 0 0 0 0 0 0 0 5",
 '8': "Don't know"}

In [27]:
import os

# Define the directory containing the files
directory = r'C:\Users\justi\Downloads\for_answers'

def process_file(file_path):
    key = os.path.basename(file_path)[:-4]  # Extract the filename without extension as key
    data = {}

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if line.startswith("Page") or "REMARKS:" in line:
                continue  # Skipping lines that start with "Page" or contain "REMARKS:"
            
            line = line.split('REMARKS:')[0]  # Remove any trailing descriptions

            parts = line.strip().split()
            for i, part in enumerate(parts):
                if part.isdigit() and int(part) in range(1, 9):
                    if part not in data:
                        label = ' '.join(filter(lambda x: not x.isdigit(), parts[:i]))
                        data[part] = label.strip()
                elif part == '9':
                    return data  # Stop processing as soon as '9' is found

    return data

def process_all_files(directory):
    result_dict = {}
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            result_dict[os.path.splitext(filename)[0]] = process_file(file_path)
            print(f"Processed {filename}")
    return result_dict

# Execute the function and store the result in a dictionary
result_dictionary = process_all_files(directory)

# Display the resulting dictionary
result_dictionary


Processed ABANY.txt
Processed ABCARE.txt
Processed ABCHOOSE.txt
Processed ABCON1.txt
Processed ABCON2.txt
Processed ABCON3.txt
Processed ABDEFCT1.txt
Processed ABDEFCTW.txt
Processed ABDEFECT.txt
Processed ABFELEGL.txt
Processed ABFIRM.txt
Processed ABHAVE1.txt
Processed ABHAVE2.txt
Processed ABHAVE3.txt
Processed ABHELP1.txt
Processed ABHELP2.txt
Processed ABHELP3.txt
Processed ABHELP4.txt
Processed ABHLTH.txt
Processed ABIMP.txt
Processed ABINFO.txt
Processed ABINSPAY.txt
Processed ABLEGAL.txt
Processed ABMEDGOV1.txt
Processed ABMEDGOV2.txt
Processed ABMELEGL.txt
Processed ABMORAL.txt
Processed ABNOMORE.txt
Processed ABORCT.txt
Processed ABORT12.txt
Processed ABPOOR.txt
Processed ABPOOR1.txt
Processed ABPOORW.txt
Processed ABPRO1.txt
Processed ABPRO2.txt
Processed ABPRO3.txt
Processed ABRAPE.txt
Processed ABROAD.txt
Processed ABSENT.txt
Processed ABSINGLE.txt
Processed ABSPNO.txt
Processed ABSTATE1.txt
Processed ABSTATE2.txt
Processed ACCNTSCI.txt
Processed ACCPTOTH.txt
Processed ACQ

{'ABANY': {'1': 'Yes', '2': 'No', '8': "Don't know"},
 'ABCARE': {'1': 'Very concerned',
  '2': 'Somewhat concerned',
  '3': 'Not very concerned',
  '4': 'Not concerned at all',
  '8': "Don't know",
  '5': "Don't know"},
 'ABCHOOSE': {'1': 'Strongly agree',
  '2': 'Agree',
  '3': 'Neither agree nor disagree',
  '4': 'Disagree',
  '5': 'Strongly disagree',
  '8': "Can't choose"},
 'ABCON1': {},
 'ABCON2': {'2': '31-40', '1': 'No answer', '3': 'No answer'},
 'ABCON3': {'5': '21-30',
  '1': '31-40',
  '8': '41-50',
  '2': 'No answer',
  '3': 'No answer'},
 'ABDEFCT1': {'1': 'Definitely should allow it',
  '2': 'Probably should allow it',
  '3': 'Probably should not allow it',
  '4': 'Definitely should not allow it',
  '8': "Can't choose"},
 'ABDEFCTW': {'1': 'Always wrong',
  '2': 'Almost always wrong',
  '3': 'Wrong only sometimes',
  '4': 'Not wrong at all',
  '8': "Can't choose"},
 'ABDEFECT': {'1': 'Yes', '2': 'No', '8': "Don't know"},
 'ABFELEGL': {'1': 'Should', '2': 'Should not', '

In [35]:
a = clean_descriptions(result_dictionary)


In [37]:
a["FUND"]

{'1': 'Fundamentalist', '2': 'Moderate', '3': 'Liberal'}

In [ ]:
app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Data Analysis Dashboard"), width={'size': 6, 'offset': 3})),
    dbc.Row([
        dbc.Col(
            dbc.Form([
                dbc.Label("Segment", html_for="segment-dropdown"),
                dcc.Dropdown(
                    id='segment-dropdown',
                    options=[{'label': key, 'value': key} for key in segment_files.keys()],
                    value='Whole Country',
                    clearable=False,
                    className='dropdown',
                    style={'width': '100%'}
                )
            ]), width=3
        ),
        dbc.Col(
            dbc.Form([
                dbc.Label("Sub-segment", html_for="sub-segment-dropdown"),
                dcc.Dropdown(
                    id='sub-segment-dropdown',
                    options=[],
                    value=None,
                    clearable=False,
                    className='dropdown',
                    style={'width': '100%'}
                )
            ]), width=3, id='sub-segment-col', style={'display': 'none'}
        ),
    ], justify='center'),
    dbc.Row([
        dbc.Col(
            dbc.Form([
                dbc.Label("Start Year", html_for="start-year-dropdown"),
                dcc.Dropdown(
                    id='start-year-dropdown',
                    options=[{'label': str(year), 'value': year} for year in range(2000, 2023)
                             if year % 2 == 0 and year != 2020 or year == 2021],
                    value='2000',
                    clearable=False,
                    className='dropdown',
                    style={'width': '100%', 'marginBottom': '0px'}  # Removes the bottom margin
                )
            ]), width=3
        ),
        dbc.Col(
            dbc.Form([
                dbc.Label("End Year", html_for="end-year-dropdown"),
                dcc.Dropdown(
                    id='end-year-dropdown',
                    options=[{'label': str(year), 'value': year} for year in range(2000, 2023)
                             if year % 2 == 0 and year != 2020 or year == 2021],
                    value='2022',
                    clearable=False,
                    className='dropdown',
                    style={'width': '100%', 'marginTop': '0px'}  # Removes the top margin
                )
            ]), width=3
        )
    ], justify='center'),
    dbc.Row(dbc.Col(
        dbc.Form([
            dbc.Label("Rows", html_for="num-rows-input"),
            dcc.Input(
                id='num-rows-input',
                type='number',
                value=10,
                min=1,
                className='input-number',
                style={'width': '100px'}
            )
        ]), width=2
    )),
    dbc.Row(dbc.Col(dcc.Loading(
        id="loading-output",
        type="default",
        children=html.Div(id="table-container")
    ), width=12)),
], fluid=True, style={'backgroundColor': '#f4f4f9'})